In [7]:
library(readr)
library(dplyr)
library(ggplot2)
library(tidyverse)
library(lubridate)
options(repr.matrix.max.rows = 7)

In [14]:
URL1 <- "https://drive.google.com/file/d/1Vk02BWAO3Xd_unJJKj0RdpTq1ciLNPO2/view?usp=sharing"

player <- read_csv(URL1)

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 133 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): <!DOCTYPE html><html><head><script nonce="gD6HICe9asqW1jLJ1hWpUw"> ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Data description - players.csv
The players.csv contains information for 196 participants with 7 columns. 

Issues with data

2 rows have Age = NA
played_hours is zero-inflated and right-skewed with large outliers
hashedEmail is a privacy-preserving unique id; name is not a reliable identifier.


Details regarding variables

experience (factor) - User competency level; one of Amateur, Beginner, Pro, Regular, Veteran.
subscribe (logical) - TRUE or FALSE
hashedEmail (chr) - Hashed email used as a unique user identifier.
played_hours (numeric) - Total hours played on the server; numeric
name (chr) - Player nickname on the server; may not be unique.
gender (factor) - Self-reported gender; values include Male, Female, Non-binary, Prefer not to say, Agender, Two-Spirited, Other.
Age (numeric) - Player age in years; numeric

In [5]:
URL2 <- "https://drive.google.com/file/d/1sgwirnt-rPXYEDSawLC2wyPBtTlJAfb1/view?usp=drive_link"

sessions <- read_csv(URL2)


Rows: 2078 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): <!DOCTYPE html><html dir="ltr"><head><script nonce="amv9YKkoKIWigk4...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Question: We are interested in demand forecasting, namely, what time windows are most likely to have large number of simultaneous players. This is because we need to ensure that the number of licenses on hand is sufficiently large to accommodate all parallel players with high probability. 

Broad Question

We aim to forecast when the server is most likely to experience high numbers of simultaneous players.

Specific Question

Can time-related factors (such as time of day, day of week, or session duration patterns) predict the number of simultaneous players within a given time window?


How the Data Help Address the Question

The dataset includes start and end timestamps for every session, which allows us to reconstruct how many users are active at any moment.

To answer the question, we will:

1.Convert the timestamp variables into proper datetime formats.
2.Divide the timeline into fixed windows (e.g., 15-minute or 1-hour intervals).
3.Count how many sessions overlap with each time window to obtain concurrent player counts.
4.Create explanatory variables such as:
  hour of day
  day of week
  weekday vs. weekend
  average session duration

Treat concurrency in each window as the response variable, and the time-based features as explanatory variables.

ERROR: Error in eval(expr, envir, enclos): object 'player' not found
